In [1]:
# to access root modules
if __name__ == "__main__" and __package__ is None:
    from sys import path
    from os.path import dirname as dir
    path.append(dir(path[0])+'/dash_app')
    __package__ = "dash_app"

In [33]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from charts import stations_map, map_columns
from settings import *
import time

In [ ]:
data_path = DATA_PATH+'Atmospheric_Domain/2.1SurfaceAirTemperature/Figure2.1/'
data_csv = data_path + 'Figure2.1_data.csv'
"""
Tidy data for charts
"""
xls = pd.ExcelFile(
    data_path+'AnnualMeanSurfaceAirTemperature1900-2019.xlsx')
original_df = pd.read_excel(xls, 'Sheet1',skiprows=[0])
columns_dict = {'datetime':'Unnamed: 0',
               'mean__annual__surface_air_temperature':'Unnamed: 6',
                'normal_1961_1990__surface_air_temperature':'Unnamed: 9',
               'mean__annual__surface_air_temperature_anomaly':'Anom',
               'moving_average__11year__surface_air_temperature_anomaly':'Unnamed: 10',
                  }
df = map_columns(columns_dict, original_df)
df['datetime'] = pd.to_datetime(df['datetime'],format='%Y')
df['location'] = 'Ireland'
df['moving_average__11year__surface_air_temperature']=df['moving_average__11year__surface_air_temperature_anomaly']+9.55
df.dropna(subset=['mean__annual__surface_air_temperature'], inplace=True)
# df.to_csv(data_csv)

In [ ]:

"""
Read from csv
"""
data_path = DATA_PATH+'Atmospheric_Domain/2.1SurfaceAirTemperature/Figure2.1/'
data_csv = data_path + 'Figure2.1_data.csv'
start_time = time.time()
df = pd.read_csv(data_csv, index_col=0)
df['datetime'] = pd.to_datetime(df['datetime'])
annualTrace = go.Scatter(x=df.datetime,
                         y=df.mean__annual__surface_air_temperature,
                         name='Annual Mean',
                         mode='markers',
                         text=df.mean__annual__surface_air_temperature_anomaly,
                         marker=dict(color=TIMESERIES_COLOR_2,
                                     size=5,
                                     opacity=0.5),
                         hovertemplate='%{x|%Y}<br>' +
                         '<b>Annual</b><br>' +
                         'Tmean: %{y:.2f} \u00b0C<br>' +
                         'Anomaly: %{text:.2f} \u00b0C<extra></extra>'
                         )

movingAvgTrace = go.Scatter(x=df.datetime,
                            y=df.moving_average__11year__surface_air_temperature_anomaly,
                            text=df.moving_average__11year__surface_air_temperature,
                            name='11 Year Moving Average',
                            mode='lines',  # 'line' is default
                            line_shape='spline',
                            line=dict(color=TIMESERIES_COLOR_1,
                                      width=2),
                            hovertemplate='%{x|%Y}<br>' +
                            '<b>11 Year Moving Average</b><br>' +
                            'Tmean: %{text:.2f} \u00b0C<br>' +
                            'Anomaly: %{y:.2f} \u00b0C<extra></extra>'
                            )
movingAvgDF=df
movingAvgDF.dropna(subset=['moving_average__11year__surface_air_temperature_anomaly'],inplace=True)
# movingAvgDF = df.loc[df.moving_average__11year__surface_air_temperature_anomaly.notna()]
movingAvgDF['year'] = movingAvgDF['datetime'].dt.year
linearTrendPoly = np.polyfit(
    movingAvgDF['year'], movingAvgDF['moving_average__11year__surface_air_temperature_anomaly'],1)
linearTrendY = np.poly1d(linearTrendPoly)(movingAvgDF['year'])
linearTrendTrace = go.Scatter(x=movingAvgDF['datetime'],
                              y=linearTrendY,
                              name='Linear Trend',
                              line=dict(color=TIMESERIES_COLOR_1,
                                        dash='dash',
                                        width=2),
                              hoverinfo='skip',
                              )
# normal_value = df.normal_1961_1990__surface_air_temperature[0]
normal = go.Scatter(x=['1898-01-01', '2021-01-01'],
                    y=[9.55,9.55],
                    name='1961-1990 Normal',
                    mode='lines',  # 'line' is default
                    line_shape='spline',
                    line=dict(color=TIMESERIES_COLOR_3, 
                              width=2),
                    hoverinfo='skip',
                    )

figure_2_1 = make_subplots(specs=[[{'secondary_y': True}]])

figure_2_1.add_trace(annualTrace,
                     secondary_y=True,)
figure_2_1.add_trace(movingAvgTrace,
                     secondary_y=False)
figure_2_1.add_trace(linearTrendTrace,
                     secondary_y=False)
figure_2_1.add_trace(normal,
                     secondary_y=True)
figure_2_1.update_layout(TIMESERIES_LAYOUT)
# Update y-axes layout seperatly due to the double y-axis chart
figure_2_1.update_yaxes(title_text='Difference (\u00b0C) from 1961-1990 Normal',
                        secondary_y=False,
                        range=[-0.9, 1.3],
                        showgrid=False,
                        dtick=0.5,  # dtick sets the distance between ticks
                        tick0=0,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        showspikes=True,
                        zeroline=True,  # add a zero line
#                         zerolinecolor=TIMESERIES_COLOR_3
                        )

figure_2_1.update_yaxes(title_text='Mean Annual Temperature (\u00b0C)',
                        secondary_y=True,
                        range=[8.65, 10.85],
                        showgrid=False,
                        dtick=0.5,  # dtick sets the distance between ticks
                        tick0=9.55,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        #                      linewidth = 2,
                        #                   spikethickness = 2,
                        #                   linecolor = '#356b6a'
                        )

figure_2_1.update_xaxes(
    title_standoff=0.1,
    title_text='Year',
    range=['1898-01-01', '2021-01-01'],
    fixedrange=True,  # stops the users being able to zoom
#     tickformat='000',  # number format
    showspikes=True,  # show the spike lines on hover
    spikethickness=2,  # spike line thickness
    #                  linewidth = 2, #width of axis line
    #                  linecolor = '#356b6a'
)  # colour of axis line

figure_2_1.add_annotation(x='2010-01-01',
                          y=0.5,
                          text='1961-1990 Normal',
                          showarrow=False,
                          font=dict(
                              color=TIMESERIES_COLOR_3),)
print("--- %s seconds ---" % (time.time() - start_time))
figure_2_1

In [5]:
# used for static images on main site
figure_2_1.update_layout(
    margin={"t": 40, "b": 20, "r": 40, "l": 80, },
    plot_bgcolor='#f7fbfd',
    paper_bgcolor='white',
    height=500,
    width=900,
    font_size=14,
    title=dict(
        text='<b>Mean Annual Surface Air Temperature - Ireland (1900-2019)</b>',
        x=0.5,
        y=0.97),)
figure_2_1.update_yaxes(
    secondary_y=False,
    title_standoff=0.2
)
figure_2_1.update_xaxes(
    title_standoff=0.1
)
figure_2_1.write_image(
    "/Users/dan/ClimateIreland/CI-Status-Report/images/mean__annual__surface_air_temperature.png", 
    format="png"
    )

In [37]:
"""
No longer used. Read from excel
"""


start_time = time.time()
data_path = DATA_PATH+'Atmospheric_Domain/2.1SurfaceAirTemperature/Figure2.1/'
xls = pd.ExcelFile(
    data_path+'AnnualMeanSurfaceAirTemperature1900-2019.xlsx')
dataDF = pd.read_excel(xls, 'Sheet1')
dataDF = dataDF[1:]
dataDF = dataDF.rename(columns={'1961-1990 Normal': 'Anom'})

movingAvgTrace = go.Scatter(x=dataDF.Year,
                            y=dataDF.filter11,
                            text=dataDF['Std Dev (11 year average)'],
                            name='11 Year Moving Average',
                            mode='lines',  # 'line' is default
                            line_shape='spline',
                            line=dict(color=TIMESERIES_COLOR_1,
                                      width=2),
                            hovertemplate='%{x}<br>' +
                            '<b>Moving Average</b><br>' +
                            'Anomaly: %{y:.2f} \u00b0C<extra></extra>'
                            )
annualTrace = go.Scatter(x=dataDF.Year,
                         y=dataDF.Tmean,
                         name='Annual Mean',
                         mode='markers',
                         text=dataDF.Anom,
                         marker=dict(color=TIMESERIES_COLOR_2,
                                     size=5,
                                     opacity=0.5),
                         hovertemplate='%{x}<br>' +
                         '<b>Annual</b><br>' +
                         'Tmean: %{y:.2f} \u00b0C<br>' +
                         'Anomaly: %{text:.2f} \u00b0C<extra></extra>'
                         )

movingAvgDF = dataDF.loc[dataDF.filter11.notna()]
linearTrendPoly = np.polyfit(
    movingAvgDF['Year'], movingAvgDF['filter11'], 1)
linearTrendY = np.poly1d(linearTrendPoly)(movingAvgDF['Year'])
linearTrendTrace = go.Scatter(x=movingAvgDF['Year'],
                              y=linearTrendY,
                              name='Linear Trend',
                              line=dict(color=TIMESERIES_COLOR_1,
                                        dash='dash',
                                        width=2),
                              hoverinfo='skip',
                              )

figure_2_1 = make_subplots(specs=[[{'secondary_y': True}]])

figure_2_1.add_trace(annualTrace,
                     secondary_y=True,)
figure_2_1.add_trace(movingAvgTrace,
                     secondary_y=False)
figure_2_1.add_trace(linearTrendTrace,
                     secondary_y=False)
figure_2_1.update_layout(TIMESERIES_LAYOUT)
# Update y-axes layout seperatly due to the double y-axis chart
figure_2_1.update_yaxes(title_text='Difference (\u00b0C) from 1961-1990 Normal',
                        secondary_y=False,
                        range=[-0.9, 1.3],
                        showgrid=False,
                        dtick=0.5,  # dtick sets the distance between ticks
                        tick0=0,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        showspikes=True,
                        zeroline=True,  # add a zero line
                        zerolinecolor=TIMESERIES_COLOR_3
                        )

figure_2_1.update_yaxes(title_text='Mean Annual Temperature (\u00b0C)',
                        secondary_y=True,
                        range=[8.65, 10.85],
                        showgrid=False,
                        dtick=0.5,  # dtick sets the distance between ticks
                        tick0=9.55,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        #                      linewidth = 2,
                        #                   spikethickness = 2,
                        #                   linecolor = '#356b6a'
                        )

figure_2_1.update_xaxes(
    title='Year',
    fixedrange=True,  # stops the users being able to zoom
    tickformat='000',  # number format
    showspikes=True,  # show the spike lines on hover
    spikethickness=2,  # spike line thickness
    #                  linewidth = 2, #width of axis line
    #                  linecolor = '#356b6a'
)  # colour of axis line

figure_2_1.add_annotation(x=2015,
                          y=0.053,
                          text='1961-1990 Normal',
                          showarrow=False,
                          font=dict(
                              color=TIMESERIES_COLOR_3),)
print("--- %s seconds ---" % (time.time() - start_time))
figure_2_1

--- 0.15375375747680664 seconds ---


In [ ]:
"""
Wet/dry days per decade
"""
data_path = DATA_PATH+'Atmospheric_Domain/2.1SurfaceAirTemperature/Figure2.3/'
df = pd.read_csv(data_path+'Fig2.3_StationsTable.txt', delimiter = ",")
df=df.round(2) 
df

In [ ]:
csd1DF = df.loc[df['CSDI_days_']<= 0]
csd1DFStr=csd1DF.astype(str)
csd1Trend = go.Scattermapbox(
        name='<= 0.0',
        lon=csd1DF.lon,
        lat=csd1DF.lat,
        marker=dict(color="#4ce600",#green
                    size=8,),
        hovertemplate='<b>'+csd1DF["station_id"]+'</b><br>' +
       'Trend in CSD per Decade: ' + csd1DFStr["CSDI_days_"]+ ' days<br>' +
        'Lat: ' + csd1DFStr["lat"]+ '\u00b0<br>' +
        'Lon: ' + csd1DFStr["lon"]+ '\u00b0<br>' +
        '<extra><br>'+
        'Trend in WSD per Decade: ' + csd1DFStr["WSDI_days_"]+' days<br>' +
        '</extra>',
    )

csd2DF = df.loc[(df['CSDI_days_']>0)&(df['CSDI_days_']<=1.0)]
csd2DFStr=csd2DF.astype(str)
csd2Trend = go.Scattermapbox(
        name='> 0.0, <= 1.0',
        lon=csd2DF.lon,
        lat=csd2DF.lat,
        marker=dict(color="#004da8",#blue
                    size=10,),
        hovertemplate='<b>'+csd2DFStr["station_id"]+'</b><br>' +
       'Trend in CSD per Decade: ' + csd2DFStr["CSDI_days_"]+' days<br>' +
        'Lat: ' + csd2DFStr["lat"]+ '\u00b0<br>' +
        'Lon: ' + csd2DFStr["lon"]+ '\u00b0<br>' +
        'Trend in WSD per Decade: ' + csd2DFStr["WSDI_days_"]+' days<br>' +
        '<extra></extra>',
    )
csd3DF = df.loc[(df['CSDI_days_']>1.0)&(df['CSDI_days_']<=2.0)]
csd3DFStr=csd3DF.astype(str)
csd3Trend = go.Scattermapbox(
        name='> 1.0, <= 2.0',
        lon=csd3DF.lon,
        lat=csd3DF.lat,
        marker=dict(color="#a900e6",#pink
                    size=12,),
        hovertemplate='<b>'+csd3DFStr["station_id"]+'</b><br>' +
       'Trend in CSD per Decade: ' + csd3DFStr["CSDI_days_"]+' days<br>' +
        'Lat: ' + csd3DFStr["lat"]+ '\u00b0<br>' +
        'Lon: ' + csd3DFStr["lon"]+ '\u00b0<br>' +
        'Trend in WSD per Decade: ' + csd3DFStr["WSDI_days_"]+' days<br>' +
        '<extra></extra>',
    )
csd4DF = df.loc[df['CSDI_days_']>2.0]
csd4DFStr=csd4DF.astype(str)
csd4Trend = go.Scattermapbox(
        name='> 2.0',
        lon=csd4DF.lon,
        lat=csd4DF.lat,
        marker=dict(color="#4c0073",#purple
                    size=14,),
        hovertemplate='<b>'+csd4DFStr["station_id"]+'</b><br>' +
       'Trend in CSD per Decade: ' + csd4DFStr["CSDI_days_"]+' days<br>' +
        'Lat: ' + csd4DFStr["lat"]+ '\u00b0<br>' +
        'Lon: ' + csd4DFStr["lon"]+ '\u00b0<br>' +
        'Trend in WSD per Decade: ' + csd4DFStr["WSDI_days_"]+' days<br>' +
        '<extra></extra>',
    )

In [ ]:
figure_2_3_1 = go.Figure(
        data=[csd1Trend,csd2Trend,csd3Trend,csd4Trend],
        layout=MAP_LAYOUT)
figure_2_3_1.update_layout(legend_title="<b>Trend in number of"+
                      "<br>annual cold spell days (CSD) per decade</b>")
figure_2_3_1

In [ ]:
# wsd1DF = df.loc[(df['WSDI_days_']<=0.0)]
# wsd1DFStr=wsd1DF.astype(str)
# wsd1Trend = go.Scattermapbox(
#         name='<= 0.0',
#         lon=wsd1DF.lon,
#         lat=wsd1DF.lat,
#         marker=dict(color="#70a800",#green
#                     size=8,),
#         hovertemplate='<b>'+wsd1DF["station_id"]+'</b><br>' +
#        'Trend in WSDI per Decade: ' + wsd1DFStr["WSDI_days_"]+ ' days<br>' +
#         'Lat: ' + wsd1DFStr["lat"]+ '\u00b0<br>' +
#         'Lon: ' + wsd1DFStr["lon"]+ '\u00b0<br>' +
#         '<extra><br>'+
#         'Trend in CSDI per Decade: ' + wsd1DFStr["CSDI_days_"]+' days<br>' +
#         '</extra>',
#     )
wsd1Trend = go.Scattermapbox(
        name='<= 0.0',
        lon=[None],
        lat=[None],
        marker=dict(color="#70a800",#green
                    size=8,),
    )

wsd2DF = df.loc[(df['WSDI_days_']>0.0)&(df['WSDI_days_']<=1.0)]
wsd2DFStr=wsd2DF.astype(str)
wsd2Trend = go.Scattermapbox(
        name='> 0.0, <= 1.0',
        lon=wsd2DF.lon,
        lat=wsd2DF.lat,
        marker=dict(color="#ffff00",#yellow
                    size=10,),
        hovertemplate='<b>'+wsd2DFStr["station_id"]+'</b><br>' +
       'Trend in WSD per Decade: ' + wsd2DFStr["WSDI_days_"]+' days<br>' +
        'Lat: ' + wsd2DFStr["lat"]+ '\u00b0<br>' +
        'Lon: ' + wsd2DFStr["lon"]+ '\u00b0<br>' +
        'Trend in CSD per Decade: ' + wsd2DFStr["CSDI_days_"]+' days<br>' +
        '<extra></extra>',
    )
wsd3DF = df.loc[(df['WSDI_days_']>1.0)&(df['WSDI_days_']<=2.0)]
wsd3DFStr=wsd3DF.astype(str)
wsd3Trend = go.Scattermapbox(
        name='> 1.0, <= 2.0',
        lon=wsd3DF.lon,
        lat=wsd3DF.lat,
        marker=dict(color="#e69800",#orange
                    size=12,),
        hovertemplate='<b>'+wsd3DFStr["station_id"]+'</b><br>' +
       'Trend in WSD per Decade: ' + wsd3DFStr["WSDI_days_"]+' days<br>' +
        'Lat: ' + wsd3DFStr["lat"]+ '\u00b0<br>' +
        'Lon: ' + wsd3DFStr["lon"]+ '\u00b0<br>' +
        'Trend in CSD per Decade: ' + wsd3DFStr["CSDI_days_"]+' days<br>' +
        '<extra></extra>',
    )
wsd4DF = df.loc[(df['WSDI_days_']>2.0)]
wsd4DFStr=wsd4DF.astype(str)
wsd4Trend = go.Scattermapbox(
        name='> 2.0',
        lon=wsd4DF.lon,
        lat=wsd4DF.lat,
        marker=dict(color="#e60000",#red
                    size=14,),
        hovertemplate='<b>'+wsd4DFStr["station_id"]+'</b><br>' +
       'Trend in WSD per Decade: ' + wsd4DFStr["WSDI_days_"]+' days<br>' +
        'Lat: ' + wsd4DFStr["lat"]+ '\u00b0<br>' +
        'Lon: ' + wsd4DFStr["lon"]+ '\u00b0<br>' +
        'Trend in CSD per Decade: ' + wsd4DFStr["CSDI_days_"]+' days<br>' +
        '<extra></extra>',
    )

In [ ]:
figure_2_3_2 = go.Figure(
        data=[wsd1Trend,wsd2Trend,wsd3Trend,wsd4Trend],
        layout=MAP_LAYOUT)
figure_2_3_2.update_layout(legend_title="<b>Trend in number of"+
                      "<br>annual warm spell days (WSD) per decade</b>")
figure_2_3_2